### 5-12. 저장된 모델 불러오기

힘들게 학습시킨 모델을 불러오겠습니다.
- 제대로 불러오는지 확인하기 위해, [런타임 연결 해제 및 삭제] 한다.
- Runtime을 새로 연결해주세요.
- 필요한 Library를 처음부터 다시 설치할겁니다.

In [ ]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
   # Colab은 아래 코드가 수행됩니다.
    import torch; v = re.match(r"[0-9\.]{3,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.32.post2" if v == "2.8.0" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2

이전에 저장했던 'gemma_jeju_lora' 폴더를 Colab에 업로드 합니다.
- 'gemma_jeju_lora' 폴더 생성하기
- 생성한 폴더에 이전에 저장했던 파일들을 끌어다 놓기

- **LoRA 어댑터만 저장한 경우**: 아래 명령어를 실행해 불러옵니다.

In [ ]:
from unsloth import FastLanguageModel

# Colab 기준 2분 소요
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="gemma_jeju_lora",  # 저장한 LoRA 어댑터 경로
    max_seq_length=2048,
    load_in_4bit=True,
)

불러온 모델을 이용해 추론해봅시다.

In [ ]:
messages = [
    {"role" : "user", "content" : "다음 문장을 제주도 사투리로 번역해줘:\n\n 귤 먹으면서 바다에서 놀면 좋아요!"}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 1000, # Increase for longer outputs!
    temperature = 1.4, top_p = 0.9, top_k = 20, # For non thinking
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)